<a href="https://placekey.io/">
         <img src="https://assets.website-files.com/5f08ccbb93b299154d34ef7f/5f1e9c2bdd643e0d63967ca9_logo-placekey-black.svg" alt="Placekey" ALIGN="left" width=600>
      </a>



# Quick Start Guide For Batch Process [Placekey](https://placekey.io/) Matching With Python
--------------
**[Ryan Fox Squire](https://www.linkedin.com/in/ryanfoxsquire/) | Senior Data Scientist, [SafeGraph](https://safegraph.com/)**

 <img src="https://global-uploads.webflow.com/5baafc2653bd67278f206724/5be267a03f7813daf821b31e_safegraph-logo-hidpi%403x-p-500.png" alt="SafeGraph" ALIGN="center" width=100>


Sep 2020

--------------
*Share this notebook: [Shareable Link](https://colab.research.google.com/drive/1Cg89DtBv2hfpuQhR0G0zvA0ylmQqEVwr?usp=sharing)*



## Summary
* [Placekey](https://placekey.io/) is geospatially-aware universal place identifier useful for joining together disparate geospatial, address, and point-of-interest datasets. 
* Placekey is an open and free to use.
* Here we demonstrate how to obtain Placekeys for point-of-interest or address-based data using the Placekey API and python.


#### What do you need? 
* Your Placekey API key --> [Register Here](https://dev.placekey.io/default/register) 
* Your dataset containing points-of-interest, addresses, and/or geospatial coordinates for which you want to assign a universal place identifier (placekey)


#### Have questions or ideas? 
 * Join thousands of geospatial data scientists in the [Placekey Slack Community](https://www.placekey.io/community) to get help and   share ideas.
 * Follow [Placekey on Github](https://github.com/Placekey)
 * Tweet [@placekeyio](https://twitter.com/placekeyio?lang=en)

In [ ]:
import pandas as pd
import numpy as np
import json

## Set Your API Key

Get your Placekey API instantly here: [Register for Placekey API](https://dev.placekey.io/default/register) 


In [ ]:
api_key = "ENTER YOUR API KEY HERE" # fill this in with your personal API key (do not share publicly)

# api_key = pd.read_csv("/content/drive/My Drive/demo_keys/squire_placekey_api_key").columns[0] # for original author debugging

## Read in Data
* Define `data_path` and `data_filename`
* Mind your datatypes (do not drop leading zeros for postal code)


In [ ]:
data_path = "/content/drive/My Drive/publicly_shared/CostcoWholesaleCorp-Sept-2020-CORE_POI/" 
data_filename = "my_places_data_missing"
file_suffix = ".csv"
full_path = data_path + data_filename + file_suffix
print(full_path)

/content/drive/My Drive/publicly_shared/CostcoWholesaleCorp-Sept-2020-CORE_POI/my_places_data_missing.csv


In [ ]:
my_dtypes = {'internal_place_id':str, 
             'store_name' : str,
             'street_address':str, 
             'city': str, 
             'state': str, 
             'zip_code': str, # This is important so that you do not drop leading zeros
             'iso_country_code':str,
             'lat' : float,
             'lon' : float}
orig_df = pd.read_csv(full_path, dtype=my_dtypes)
print(orig_df.shape)
orig_df.head()

(545, 8)


,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


## Prep Data Column Names for Placekey API

https://docs.placekey.io/

Make sure to list every column useful for the Placekey API in `column_map` even if you do not need to rename it (see `city` in the example).

In [ ]:
query_id_col = "internal_place_id" # this column in your data should be unique for every row
column_map = {query_id_col: "query_id",
              "store_name" : "location_name",
              "full_address" : "street_address",
              "city": "city",
              "state": "region",
              "zip_code": "postal_code",
              "lat": "latitude",
              "lon": "longitude"
             }

df_for_api = orig_df.rename(columns=column_map)
cols = list(column_map.values())
df_for_api = df_for_api[cols]

# add missing hard-coded columns
df_for_api['iso_country_code'] = 'US'
df_for_api.head()

,query_id,location_name,street_address,city,region,postal_code,latitude,longitude,iso_country_code
0,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,5401 Katella Ave,Cypress,CA,90720,33.804504,-118.039593,US
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,25900 SW Heather Pl,Wilsonville,OR,NaN,NaN,NaN,US
2,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,400 Costco Dr Ste 150,Tukwila,WA,98188,47.445371,-122.248623,US
3,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,2500 Harlem Ave,North Riverside,IL,60546,41.844777,-87.806473,US
4,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,4200 Rusty Rd,Concord,MO,63128,38.508769,-90.338324,US


## Data Cleaning Best Practices

Gotcha's to check for:
* ""
* " "
* "null"/"Null"
* "None"
* Missing / Nan
* Anything else weird in your data? 
* Convert all of these to `np.nan` so that they are handled correctly by `json` library

In [ ]:
df_clean = df_for_api.copy()
possible_bad_values = ["", " ", "null", "Null", "None", "nan", "Nan"] # Any other dirty data you need to clean up? 
for bad_value in possible_bad_values:
  df_clean = df_clean.replace(to_replace=bad_value, value=np.nan)

print("FYI data missing from at least 1 column in the following number of rows:")
print(df_clean.shape[0] - df_clean.dropna().shape[0])
print("Some examples of rows with missing data")
df_clean[df_clean.isnull().any(axis=1)].head()

FYI data missing from at least 1 column in the following number of rows:
18
Some examples of rows with missing data


,query_id,location_name,street_address,city,region,postal_code,latitude,longitude,iso_country_code
1,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,25900 SW Heather Pl,Wilsonville,OR,NaN,NaN,NaN,US
5,sg:03cb6aeb0e024bd6bf71626e30b201a0,Costco Wholesale Corp.,5901 Redwood Dr,Rohnert Park,CA,94928,NaN,NaN,US
10,sg:377f1c2a3c3e4b0885b8addcb4f70997,Costco Wholesale Corp.,1501 W Kelly Ave,Pharr,TX,78577,NaN,NaN,US
12,sg:98ccf43c4a934f30a943a9564ce4cf0e,Costco Wholesale Corp.,24008 Snohomish Woodinville Rd,Maltby,WA,NaN,47.779382,-122.148732,US
16,sg:be95a4498ee340db9b5eeb3130124eab,Costco Wholesale Corp.,1415 N Arizona Ave,Gilbert,AZ,NaN,33.376033,-111.840291,US


In this example example dataset some rows are missing `postal_code`, `latitude`/`longitude`, and even the `street_address`. 

The Placekey API is flexible and powerful and can often handle incomplete data. We will send the API the fields we have and see what comes back. 

### Prepare Data Structure For Batch API 

https://docs.placekey.io/#5778b1b7-ce57-4cbf-a74e-bf4e2ce5a9ec

Advanced: If you want to include options like [`strict_address_matching`](https://docs.placekey.io/#77042b13-ad19-423b-b117-c1ec819ae2f9) for more conservative results, edit the function `prepare_batches_for_API`. 

In [ ]:
import http.client

headers = {
    'accept': "application/json",
    'apikey': api_key,
    'content-type': "application/json"
    }

def prepare_batches_for_API(all_batches):
    batch_payloads = []
    for batch in all_batches:
        payload = json.dumps({"queries":batch, 
                              "options": {"strict_address_match": False}}) #see https://docs.placekey.io/#77042b13-ad19-423b-b117-c1ec819ae2f9
        batch_payloads.append(payload)
    return batch_payloads

def getPlacekeys(batch_payloads, debug=False):
    conn = http.client.HTTPSConnection("api.placekey.io") 
    responses = []
    for payload in batch_payloads:
        conn.request("POST", "/v1/placekeys", payload, headers)
        res = conn.getresponse()
        data = res.read()
        if(debug):   
          print("data", data)
          print("status", res.status)
          print("headers", res.getheaders())
        responses.append(json.loads(data))
    return responses

In [ ]:
data_jsoned = json.loads(df_clean.to_json(orient="records"))
print(len(data_jsoned))
data_jsoned[0]

545


{'city': 'Cypress',
 'iso_country_code': 'US',
 'latitude': 33.804504,
 'location_name': 'Costco Wholesale Corp.',
 'longitude': -118.039593,
 'postal_code': '90720',
 'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3',
 'region': 'CA',
 'street_address': '5401 Katella Ave'}

In [ ]:
# Chunk the data into small batches
max_batch_size = 50 
batches = [data_jsoned[i:i + max_batch_size] for i in range(0, len(data_jsoned), max_batch_size)]
batches_json = prepare_batches_for_API(batches)
print("Number of batches:" ,len(batches_json), "\n")
batches_json[0]

Number of batches: 11 



'{"queries": [{"query_id": "sg:22a7df9c14c148bb969171d93e0950c3", "location_name": "Costco Wholesale Corp.", "street_address": "5401 Katella Ave", "city": "Cypress", "region": "CA", "postal_code": "90720", "latitude": 33.804504, "longitude": -118.039593, "iso_country_code": "US"}, {"query_id": "sg:235e4067e4b04332b23e5a38448f6535", "location_name": "Costco Wholesale Corp.", "street_address": "25900 SW Heather Pl", "city": "Wilsonville", "region": "OR", "postal_code": null, "latitude": null, "longitude": null, "iso_country_code": "US"}, {"query_id": "sg:67591b2e593048a4bd4e09e027ba9caf", "location_name": "Costco Wholesale Corp.", "street_address": "400 Costco Dr Ste 150", "city": "Tukwila", "region": "WA", "postal_code": "98188", "latitude": 47.445371, "longitude": -122.248623, "iso_country_code": "US"}, {"query_id": "sg:b568fdaf8fd64651ad1f5931e48440fe", "location_name": "Costco Wholesale Corp.", "street_address": "2500 Harlem Ave", "city": "North Riverside", "region": "IL", "postal_co

## Request Placekeys From Placekey API and View API Results

In [ ]:
%%time
responses = getPlacekeys(batches_json)


CPU times: user 18.2 ms, sys: 2.94 ms, total: 21.2 ms
Wall time: 5.81 s


In [ ]:
print("total batches sent:", len(batches_json))
print("total batches returned:", len(responses))
print("length of individual batches:", len(responses[0]))
print("example individual query response:")
print(responses[0][0])

total batches sent: 11
total batches returned: 11
length of individual batches: 50
example individual query response:
{'query_id': 'sg:22a7df9c14c148bb969171d93e0950c3', 'placekey': 'zzw-225@5z6-3p2-v9f'}


In [ ]:
# un-nest the queries from their batches
responses_flat = [item for sublist in responses for item in sublist]
print("total individual queries returned:", len(responses_flat))

# filter out invalid responses
responses_flat_cleaned = [resp for resp in responses_flat if 'query_id' in resp]
print("total successful query responses:", len(responses_flat_cleaned))

total individual queries returned: 545
total successful query responses: 545


In [ ]:
df_placekeys = pd.read_json(json.dumps(responses_flat_cleaned), dtype={'query_id':str})
df_placekeys.head(10)

,query_id,placekey
0,sg:22a7df9c14c148bb969171d93e0950c3,zzw-225@5z6-3p2-v9f
1,sg:235e4067e4b04332b23e5a38448f6535,223-223@5xc-q57-xqz
2,sg:67591b2e593048a4bd4e09e027ba9caf,223-222@5x4-4sg-vfz
3,sg:b568fdaf8fd64651ad1f5931e48440fe,zzw-223@5pw-68m-4jv
4,sg:ccacecad3a01486a8873a05a1ad9db41,223-222@5pj-24r-7t9
5,sg:03cb6aeb0e024bd6bf71626e30b201a0,223-222@5vg-55k-p7q
6,sg:3381832b1ba14db09a140e9f13bb386a,zzw-223@5xd-qd6-ghq
7,sg:2f347f46f0a34017a061bdf8d7430d01,zzw-222@5qc-x2b-7nq
8,sg:20d033ca8fe44c31a02d7325ce069bd3,zzw-222@5vg-87t-psq
9,sg:57f9bfb8ae45495d97f5af97f228d143,zzw-222@63v-bky-cbk


## Add `placekey` Column Back to Original Dataset and Save Results

In [ ]:
df_join_placekey = pd.merge(orig_df, df_placekeys, left_on=query_id_col, right_on="query_id", how='left')
final_cols = list(df_placekeys.columns) + list(orig_df.columns)
df_join_placekey = df_join_placekey[final_cols]
df_join_placekey.head()

,query_id,placekey,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,sg:22a7df9c14c148bb969171d93e0950c3,zzw-225@5z6-3p2-v9f,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,sg:235e4067e4b04332b23e5a38448f6535,223-223@5xc-q57-xqz,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,sg:67591b2e593048a4bd4e09e027ba9caf,223-222@5x4-4sg-vfz,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,sg:b568fdaf8fd64651ad1f5931e48440fe,zzw-223@5pw-68m-4jv,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,sg:ccacecad3a01486a8873a05a1ad9db41,223-222@5pj-24r-7t9,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


In [ ]:
print("Summary of results:")
total_recs = df_join_placekey.shape[0]
print("total records:", total_recs)
print("records with a placekey: {0} | {1:.2f}%".format(df_join_placekey[~df_join_placekey.placekey.isnull()].shape[0],  df_join_placekey[~df_join_placekey.placekey.isnull()].shape[0]*100/total_recs))      
print("records missing a placekey: {0} | {1:.2f}%".format(df_join_placekey[df_join_placekey.placekey.isnull()].shape[0], df_join_placekey[df_join_placekey.placekey.isnull()].shape[0]*100/total_recs))
print("records missing a query_id: {0} | {1:.2f}% (invalid query)".format(df_join_placekey[df_join_placekey.query_id.isnull()].shape[0], df_join_placekey[df_join_placekey.query_id.isnull()].shape[0]*100/total_recs))



Summary of results:
total records: 545
records with a placekey: 545 | 100.00%
records missing a placekey: 0 | 0.00%
records missing a query_id: 0 | 0.00% (invalid query)


In [ ]:
output_cols = ['placekey'] + list(orig_df.columns)
output_df = df_join_placekey[output_cols]

write_path = data_path + data_filename + "_placekeys" + file_suffix
output_df.to_csv(write_path, index=False)
print("wrote to: ", write_path)
output_df.head(5)

wrote to:  /content/drive/My Drive/publicly_shared/CostcoWholesaleCorp-Sept-2020-CORE_POI/my_places_data_missing_placekeys.csv


,placekey,internal_place_id,store_name,lat,lon,full_address,city,state,zip_code
0,zzw-225@5z6-3p2-v9f,sg:22a7df9c14c148bb969171d93e0950c3,Costco Wholesale Corp.,33.804504,-118.039593,5401 Katella Ave,Cypress,CA,90720
1,223-223@5xc-q57-xqz,sg:235e4067e4b04332b23e5a38448f6535,Costco Wholesale Corp.,NaN,NaN,25900 SW Heather Pl,Wilsonville,OR,NaN
2,223-222@5x4-4sg-vfz,sg:67591b2e593048a4bd4e09e027ba9caf,Costco Wholesale Corp.,47.445371,-122.248623,400 Costco Dr Ste 150,Tukwila,WA,98188
3,zzw-223@5pw-68m-4jv,sg:b568fdaf8fd64651ad1f5931e48440fe,Costco Wholesale Corp.,41.844777,-87.806473,2500 Harlem Ave,North Riverside,IL,60546
4,223-222@5pj-24r-7t9,sg:ccacecad3a01486a8873a05a1ad9db41,Costco Wholesale Corp.,38.508769,-90.338324,4200 Rusty Rd,Concord,MO,63128


## Success!

#### Have questions or ideas? 
 * Join thousands of geospatial data scientists in the [Placekey Slack Community](https://www.placekey.io/community) to get help and   share ideas.
 * Follow [Placekey on Github](https://github.com/Placekey).
 * [@placekeyio](https://twitter.com/placekeyio?lang=en).

<br>
<br>
<br>
<br>

<a href="https://placekey.io/">
         <img src="https://assets.website-files.com/5f08ccbb93b299154d34ef7f/5f1e9c2bdd643e0d63967ca9_logo-placekey-black.svg" alt="Placekey" ALIGN="left" width=300>
      </a>

  